In [80]:
import pandas as pd
from pybaseball.lahman import *
download_lahman()
from itertools import combinations
import time
import random

df = pd.read_csv('careerteams.csv')
wardf = pd.read_csv('codes.csv')
people = people()
people['Name'] = people['nameFirst'] + " " + people['nameLast']
df = df.merge(wardf[['code','bwar']],on='code',how='left')
df = df.merge(people[['bbrefID', 'Name']], left_on='code', right_on='bbrefID', how='left')

df['Tm'] = df['Tm'].replace('SEP', 'MIL')
df['Tm'] = df['Tm'].replace('BSN', 'ATL')
df['Tm'] = df['Tm'].replace('MLN', 'ATL')
df['Tm'] = df['Tm'].replace('WSA', 'TEX')
df['Tm'] = df['Tm'].replace('ANA', 'LAA')
df['Tm'] = df['Tm'].replace('CAL', 'LAA')
df['Tm'] = df['Tm'].replace('KCA', 'OAK')
df['Tm'] = df['Tm'].replace('PHA', 'OAK')
df['Tm'] = df['Tm'].replace('MON', 'WSN')
df['Tm'] = df['Tm'].replace('FLA', 'MIA')
df['Tm'] = df['Tm'].replace('BRO', 'LAD')
df['Tm'] = df['Tm'].replace('SLB', 'BAL')
df['Tm'] = df['Tm'].replace('TBD', 'TBR')
df['Tm'] = df['Tm'].replace('NYG', 'SFG')
df['Tm'] = df['Tm'].replace('WSH', 'MIN')

df = df.drop('bbrefID', axis = 1)
tmp = df.groupby('code').Tm.nunique().reset_index()
tmp = tmp[tmp.Tm == 1]
ridlist = list(tmp.code)
df = df[~df.code.isin(ridlist)]

# Create a dictionary mapping each player to the unique teams they've played for
players_to_teams = df.groupby('code')['Tm'].unique().apply(list).to_dict()

# Sort the team lists for each player
for player, teams in players_to_teams.items():
    players_to_teams[player] = ','.join(sorted(teams))  # convert to string

# Map the sorted team lists back to the DataFrame
df['teams_played_for'] = df['code'].map(players_to_teams)

# Sort the df by bwar
df = df.sort_values('bwar',ascending=False)

# Keep the guy with the highest bwar for ever team profile 
keeplist = list(df.groupby('teams_played_for').first()['code'])

df = df[df.code.isin(keeplist)]

In [81]:
def find_minimal_players(df, iterations=10):
    # Output dictionary
    results = {}
    curr_best = 400
    
    # Create a dictionary mapping each player to the unique teams they've played for
    players_to_teams = df.groupby('code')['Tm'].unique().apply(set).to_dict()
    
    for _ in range(iterations):

        # Convert dict items to a list and shuffle it
        players_to_teams = list(players_to_teams.items())
        random.shuffle(players_to_teams)
        players_to_teams = dict(players_to_teams)

        # Create a dictionary mapping each pair of teams to the players who have played for both teams
        team_pairs_to_players = {}
        for player, teams in players_to_teams.items():
            for team_pair in combinations(teams, 2):
                sorted_pair = tuple(sorted(team_pair))
                if sorted_pair not in team_pairs_to_players:
                    team_pairs_to_players[sorted_pair] = set()
                team_pairs_to_players[sorted_pair].add(player)
    
        
        # Use a Greedy approach to select the player who covers the most uncovered team pairs
        team_pairs_covered = set()
        players_to_keep = set()


        while len(team_pairs_covered) < len(team_pairs_to_players):
            
            uncovered_pairs = list(team_pairs_to_players.keys() - team_pairs_covered)
            print(uncovered_pairs)
            
            # Shuffle the order of uncovered pairs
            random.shuffle(uncovered_pairs)

            # Select the player who covers the most uncovered team pairs
            best_player = None
            best_player_pairs = set()
            for player, teams in players_to_teams.items():
                player_pairs = set(combinations(teams, 2)) & set(uncovered_pairs)
                if (player=='doteloc01'):
                    #print(len(uncovered_pairs))
                if len(player_pairs) > len(best_player_pairs):
                    best_player = player
                    best_player_pairs = player_pairs

                    # if the current solution is already worse than the best one, stop considering more players

                    if len(players_to_keep) + 1 >= curr_best:
                        break
            
            players_to_keep.add(best_player)
            team_pairs_covered.update(best_player_pairs)
            #if best_player == 'jacksed01':
            print (best_player+'\t'+str(len(best_player_pairs)))

        # Store the result in the dictionary
        results[len(players_to_keep)] = list(players_to_keep)
        min_key = min(results.keys())
        if min_key < curr_best:
            curr_best = min_key
            print(curr_best)
    return results


IndentationError: expected an indented block (3533220363.py, line 46)

In [82]:
import random
from itertools import combinations

def find_minimal_players(df, iterations=10):
    # Output dictionary to store results
    results = {}
    curr_best = float('inf')  # Initialized to infinity for easy comparison
    
    # Create a dictionary mapping each player to the unique teams they've played for
    players_to_teams = df.groupby('code')['Tm'].unique().apply(set).to_dict()
    
    for _ in range(iterations):

        # Shuffle the players' dictionary
        shuffled_players = list(players_to_teams.keys())
        random.shuffle(shuffled_players)
        shuffled_dict = {key: players_to_teams[key] for key in shuffled_players}

        # Create a dictionary mapping each pair of teams to the players who have played for both teams
        team_pairs_to_players = {}
        for player, teams in shuffled_dict.items():
            for team_pair in combinations(teams, 2):
                sorted_pair = tuple(sorted(team_pair))
                if sorted_pair not in team_pairs_to_players:
                    team_pairs_to_players[sorted_pair] = set()
                team_pairs_to_players[sorted_pair].add(player)

        # Use a Greedy approach to select the player who covers the most uncovered team pairs
        team_pairs_covered = set()
        players_to_keep = set()

        while len(team_pairs_covered) < len(team_pairs_to_players):
            uncovered_pairs = list(team_pairs_to_players.keys() - team_pairs_covered)
            #print(uncovered_pairs)
            # Shuffle the order of uncovered pairs
            random.shuffle(uncovered_pairs)

            # Select the player who covers the most uncovered team pairs
            best_player = None
            best_player_pairs = set()
            for player, teams in shuffled_dict.items():
                player_pairs = set([tuple(sorted(p)) for p in combinations(teams, 2)]) & set(uncovered_pairs)
                if len(player_pairs) > len(best_player_pairs):
                    best_player = player
                    best_player_pairs = player_pairs

                    # if the current solution is already worse than the best one, stop considering more players
                    if len(players_to_keep) + 1 >= curr_best:
                        break
            
            players_to_keep.add(best_player)
            team_pairs_covered.update(best_player_pairs)
            #print(best_player + '\t' + str(len(best_player_pairs)))

            # Remove the best player from consideration for the next iterations
            if best_player in shuffled_dict:
                del shuffled_dict[best_player]

        # Store the result in the dictionary
        results[len(players_to_keep)] = list(players_to_keep)
        min_key = min(results.keys())
        if min_key < curr_best:
            curr_best = min_key
            print("Current best:", curr_best)

    return results


In [83]:
start_time = time.time()
results = find_minimal_players(df, iterations=100)
end_time = time.time()

elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)



Current best: 19
Elapsed time:  48.338839054107666


In [84]:
min_key = min(results.keys())
print(min_key)

19


In [85]:
filtered_df = df[df['code'].isin(results[min_key])]
player_names = filtered_df['Name'].drop_duplicates().tolist()
print(player_names)

['Nelson Cruz', 'Kevin Mitchell', 'Royce Clayton', 'LaTroy Hawkins', 'Rich Hill', 'Ruben Sierra', 'Ron Kline', 'Octavio Dotel', 'Matt Stairs', 'Terry Mulholland', 'Dennis Cook', 'Bruce Chen', 'Edwin Jackson', 'Ron Mahay', 'Jose Guillen', 'Ron Villone', 'Willie Blair', 'Rick White', 'Paul Bako']


In [28]:
players_to_teams['jacksed01']

'ARI,ATL,BAL,CHC,CHW,DET,LAD,MIA,OAK,SDP,STL,TBR,TOR,WSN'

In [16]:
filtered_df = df[df['code'].isin(results[min_key])]

In [175]:
filtered_df.to_csv('the126.csv',index=False)

In [21]:
people.columns

Index(['playerID', 'birthYear', 'birthMonth', 'birthDay', 'birthCountry',
       'birthState', 'birthCity', 'deathYear', 'deathMonth', 'deathDay',
       'deathCountry', 'deathState', 'deathCity', 'nameFirst', 'nameLast',
       'nameGiven', 'weight', 'height', 'bats', 'throws', 'debut', 'finalGame',
       'retroID', 'bbrefID', 'Name'],
      dtype='object')

In [23]:
people

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,Name
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,David Aardsma
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,2021.0,1.0,22.0,...,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01,Hank Aaron
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01,Tommie Aaron
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01,Don Aase
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01,Andy Abad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20820,zupofr01,1939.0,8.0,29.0,USA,CA,San Francisco,2005.0,3.0,25.0,...,Frank Joseph,182.0,71.0,L,R,1957-07-01,1961-05-09,zupof101,zupofr01,Frank Zupo
20821,zuvelpa01,1958.0,10.0,31.0,USA,CA,San Mateo,NaN,NaN,NaN,...,Paul,173.0,72.0,R,R,1982-09-04,1991-05-02,zuvep001,zuvelpa01,Paul Zuvella
20822,zuverge01,1924.0,8.0,20.0,USA,MI,Holland,2014.0,9.0,8.0,...,George,195.0,76.0,R,R,1951-04-21,1959-06-15,zuveg101,zuverge01,George Zuverink
20823,zwilldu01,1888.0,11.0,2.0,USA,MO,St. Louis,1978.0,3.0,27.0,...,Edward Harrison,160.0,66.0,L,L,1910-08-14,1916-07-12,zwild101,zwilldu01,Dutch Zwilling


In [24]:
people[people.playerID != people.bbrefID]

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,Name
206,alexasc01,1989.0,7.0,10.0,USA,CA,Santa Rosa,NaN,NaN,NaN,...,Scott Alain,195.0,74.0,L,L,2015-09-02,2023-06-18,alexs001,alexasc02,Scott Alexander
518,armstbo01,1850.0,7.0,4.0,USA,MD,Baltimore,1917.0,12.0,3.0,...,Robert Livingston,160.0,74.0,NaN,NaN,1871-06-26,1871-08-29,armsr101,armstsa01,Robert Armstrong
547,arrueer01,1990.0,3.0,25.0,Cuba,Cienfuegos,Cienfuegos,NaN,NaN,NaN,...,Barbaro Erisbel,230.0,73.0,R,R,2014-05-23,2014-09-28,arrue001,arrueba01,Erisbel Arruebarrena
778,baldwof01,1881.0,11.0,3.0,USA,MI,Carson City,1942.0,2.0,16.0,...,Orson Frederick,185.0,NaN,NaN,R,1908-09-06,1908-09-22,baldo101,baldwo.01,Orson Baldwin
1130,beaveed01,1848.0,NaN,NaN,USA,NY,Troy,NaN,NaN,NaN,...,Edward P.,138.0,68.0,NaN,NaN,1871-05-09,1872-08-22,beave101,beavee.01,Edward Beavens
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20283,wilsoja03,1990.0,7.0,29.0,USA,TN,Bartlett,NaN,NaN,NaN,...,Jacob Clinton,219.0,72.0,R,R,2021-07-10,2021-08-22,wilsj005,wilsoja04,Jacob Wilson
20390,withesh01,1972.0,9.0,21.0,USA,MI,Ann Arbor,NaN,NaN,NaN,...,Shannon Bolt,185.0,75.0,R,R,1998-09-18,1998-09-18,withs001,whitesh01,Shannon Withem
20497,woodswi01,1998.0,12.0,29.0,USA,TN,Trenton,NaN,NaN,NaN,...,William,190.0,75.0,R,R,2022-04-27,2022-05-01,woodw001,woodswi02,William Woods
20504,woodwro01,1962.0,9.0,28.0,USA,NH,Hanover,NaN,NaN,NaN,...,Robert John,185.0,75.0,R,R,1985-09-05,1988-09-26,woodb001,woodwbo01,Rob Woodward


In [27]:
df = pd.read_csv('careerteams.csv')
#people = people()
people['Name'] = people['nameFirst'] + " " + people['nameLast']
df = df.merge(people[['bbrefID', 'Name']], left_on='code', right_on='bbrefID', how='left')

In [34]:
df[df.Name.isna()]

,Year,Tm,code,bbrefID,Name
36615,1991,BAL,dela,NaN,NaN
40387,1975,HOU,dela,NaN,NaN
47188,1960,CLE,dela,NaN,NaN
47189,1961,CLE,dela,NaN,NaN
47190,1962,CLE,dela,NaN,NaN
47191,1963,CLE,dela,NaN,NaN
47192,1964,MLN,dela,NaN,NaN
47193,1965,MLN,dela,NaN,NaN
47194,1966,ATL,dela,NaN,NaN
47195,1967,ATL,dela,NaN,NaN
